In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import cv2
import keras


In [2]:
!pip install kaggle

import os
import zipfile

# Set up Kaggle API credentials
os.environ['KAGGLE_USERNAME'] = 'spikal2000'
os.environ['KAGGLE_KEY'] = '8974f6a7db0a94d996b1579c714faa18'

# Download the dataset using Kaggle API
dataset_name = 'meowmeowmeowmeowmeow/gtsrb-german-traffic-sign'
output_dir = '/content/dataset'

os.makedirs(output_dir, exist_ok=True)
os.chdir(output_dir)

!kaggle datasets download -d $dataset_name

# Unzip the downloaded dataset
with zipfile.ZipFile(os.path.join(output_dir, dataset_name.split('/')[1] + '.zip'), 'r') as zip_ref:
    zip_ref.extractall()

# Continue with the rest of the code for data preprocessing, model training, and object detection


 98% 601M/612M [00:05<00:00, 152MB/s]
100% 612M/612M [00:05<00:00, 118MB/s]


In [3]:
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, csv_file, batch_size=32, dim=(32, 32), n_channels=3, n_classes=43, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = pd.read_csv(csv_file)
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.labels) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.labels['Path'].values[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.labels))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
      """Generates data containing batch_size samples"""
      # Initialization
      X = np.empty((self.batch_size, *self.dim, self.n_channels))
      y = np.empty((self.batch_size), dtype=int)

      # Generate data
      for i, ID in enumerate(list_IDs_temp):
          # Load image
          img = cv2.imread(ID)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

          # Extract ROI and resize it
          roi = self.labels.loc[self.labels['Path'] == ID, ['Roi.X1','Roi.Y1','Roi.X2','Roi.Y2']].values[0]
          img = img[roi[1]:roi[3], roi[0]:roi[2], :]  # Extract ROI
          img = tf.image.resize(img, [self.dim[0], self.dim[1]])  # Resize the ROI to match the input shape
          img = img / 255.  # normalize pixel values to [0, 1]

          # Store samples
          X[i,] = img

          # Store class
          y[i] = self.labels.loc[self.labels['Path'] == ID, 'ClassId']

      return X, keras.utils.to_categorical(y, num_classes=self.n_classes)



In [4]:
import numpy as np

In [5]:
# Define the model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [14]:
# Define parameters
params = {'dim': (32, 32),
          'batch_size': 65,
          'n_classes': 43,
          'n_channels': 3,
          'shuffle': True}

# Create generators
training_generator = DataGenerator('/content/dataset/Train.csv', **params)
validation_generator = DataGenerator('/content/dataset/Test.csv', **params)

# Train model
model.fit(training_generator, validation_data=validation_generator, epochs=55)


Epoch 1/55
603/603 [==============================] - 391s 649ms/step - loss: 0.0268 - accuracy: 0.9917 - val_loss: 0.1248 - val_accuracy: 0.9729
Epoch 2/55
603/603 [==============================] - 402s 668ms/step - loss: 0.0267 - accuracy: 0.9916 - val_loss: 0.0890 - val_accuracy: 0.9789
Epoch 3/55
603/603 [==============================] - 395s 655ms/step - loss: 0.0230 - accuracy: 0.9930 - val_loss: 0.0851 - val_accuracy: 0.9769
Epoch 4/55
603/603 [==============================] - 393s 652ms/step - loss: 0.0260 - accuracy: 0.9926 - val_loss: 0.0750 - val_accuracy: 0.9823
Epoch 5/55
603/603 [==============================] - 391s 649ms/step - loss: 0.0224 - accuracy: 0.9927 - val_loss: 0.0729 - val_accuracy: 0.9817
Epoch 6/55
603/603 [==============================] - 389s 645ms/step - loss: 0.0215 - accuracy: 0.9929 - val_loss: 0.1243 - val_accuracy: 0.9772
Epoch 7/55
603/603 [==============================] - 391s 649ms/step - loss: 0.0197 - accuracy: 0.9937 - val_loss: 0.0999 -

In [16]:
model.save('/content/drive/MyDrive/Trained Models/new_fresh_model_signs.h5')
